In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import os
from keras.preprocessing.image import Iterator,load_img
from keras import layers
from keras import models
from keras import optimizers

# iterador 
class ValIterator(Iterator):
    """Class to iterate A and B images at the same time."""

    def __init__(
                 self, 
                 dir_dataset, 
                 csv_v_filename_prefix = "Fovea_locations_v.csv", 
                 dir_Validation ="Validation", 
                 dir_Glaucoma = "Glaucoma",
                 dir_NonGlaucoma = "Non-Glaucoma",
                 target_size=(512,512),
                 batch_size=32, 
                 shuffle=True, 
                 seed = None
                ):
        
        # Carrega nome ficheiros das imagens de retina
        ## Validation
        path_Validation = os.path.join(dir_dataset, dir_Validation)
        
        self.ficheiros_Validation = set(x for x in os.listdir(path_Validation))
        print ("Numero de ficheiros Validation:", len(self.ficheiros_Validation))
        
        self.n=len(self.ficheiros_Validation)
            
        # Carega ficheiro CSV com posicao da Fove
        ## Validation
        
        path_csv_v = os.path.join(dir_dataset, csv_v_filename_prefix)

        self.data_Validation = pd.read_csv(path_csv_v)
        
        self.Fovea_X_Validation=self.data_Validation.iloc[:,3]
        self.Fovea_Y_Validation=self.data_Validation.iloc[:,4]
        
        super(ValIterator, self).__init__(self.n, batch_size,
                                               shuffle, seed)
     
# ============================================================================
# ============================================================================
#  Le um par de imagens para a memoria
# ============================================================================
    def load_imgs(self, idx):
        """Get a pair of images with index idx."""
        a_fname = self.filenames[idx]
       
        a = load_img(os.path.join(self.a_dir, a_fname),
                     grayscale=self.is_a_grayscale,
                     target_size=self.target_size)
        
        a = np.array(a)
        
        return a
    
    def _load_imgs_to_memory(self):
        """Load images to memory."""
        
        self.a = np.zeros((self.N,) + self.image_shape_a)
    
        for idx in range(self.N):
            ai = self.load_imgs(idx, False)
            self.a[idx] = ai
  
    def next(self):
        """Get the next pair of the sequence."""

        # Lock the iterator when the index is changed.
        with self.lock:
            index_array, _, current_batch_size = next(self.index_generator)
            
            batch_a = np.zeros((current_batch_size,) + self.image_shape)
#            batch_filename = ["","","",""]

        for i, j in enumerate(index_array):
            a_img= self.load_imgs(j)
            
            batch_a[i] = a_img
            
           
        batch_b=self.Fovea_X_Validation
        batch_c=self.Fovea_Y_Validation 
        
        return [batch_a,batch_b,batch_c]

# ============================================================================
# ============================================================================
#   
# ============================================================================
    def generator(self):
        while 1:
            x,y,z = self.next()
            #print x.shape,y.shape
            yield x,y,z
            
class TrainIterator(Iterator):
    """Class to iterate A and B images at the same time."""

    def __init__(
                 self, 
                 dir_dataset, 
                 csv_t_filename_prefix = "Fovea_locations_t.csv", 
                 dir_Validation ="Validation", 
                 dir_Training = "Training",
                 dir_Glaucoma = "Glaucoma",
                 dir_NonGlaucoma = "Non-Glaucoma",
                 target_size=(512,512),
                 batch_size=32, 
                 shuffle=True, 
                 seed = None
                ):
    
        ## Training
        path_Training = os.path.join(dir_dataset, dir_Training)
        
        self.ficheiros_Training = set(x for x in os.listdir(path_Training))
        print ("Numero de ficheiros Training:", len(self.ficheiros_Training))
        
        self.n=len(self.ficheiros_Training)
               
        # Carega ficheiro CSV com posicao da Fove
        ## Validation
        
        
        ## Training
        
        path_csv_t = os.path.join(dir_dataset,csv_t_filename_prefix)

        self.data_Training = pd.read_csv(path_csv_t)
        
        self.Fovea_X_Training=self.data_Training.iloc[:,2]
        self.Fovea_Y_Training=self.data_Training.iloc[:,3]
        
        super(TrainIterator, self).__init__(self.n, batch_size,
                                               shuffle, seed)
# ============================================================================
# ============================================================================
#  Le um par de imagens para a memoria
# ============================================================================
    def load_imgs(self, idx):
        """Get a pair of images with index idx."""
        a_fname = self.filenames[idx]
       
        a = load_img(os.path.join(self.a_dir, a_fname),
                     grayscale=self.is_a_grayscale,
                     target_size=self.target_size)
        
        a = np.array(a)
        
        return a
    
    def _load_imgs_to_memory(self):
        """Load images to memory."""
        
        self.a = np.zeros((self.N,) + self.image_shape_a)
    
        for idx in range(self.N):
            ai = self.load_imgs(idx, False)
            self.a[idx] = ai
  
    def next(self):
        """Get the next pair of the sequence."""

        # Lock the iterator when the index is changed.
        with self.lock:
            index_array, _, current_batch_size = next(self.index_generator)
            
            batch_a = np.zeros((current_batch_size,) + self.image_shape)
#            batch_filename = ["","","",""]

        for i, j in enumerate(index_array):
            a_img= self.load_imgs(j)
            
            batch_a[i] = a_img
            
        batch_b=self.Fovea_X_Training
        batch_c=self.Fovea_Y_Training   
        return [batch_a,batch_b,batch_c]

# ============================================================================
# ============================================================================
#   
# ============================================================================
    def generator(self):
        while 1:
            x,y,z = self.next()
            #print x.shape,y.shape
            yield x,y,z
            
    
            
val_it   = ValIterator(dir_dataset = r'C:\Users\Admin\Desktop\projeto\DataSet')

train_it   = TrainIterator(dir_dataset = r'C:\Users\Admin\Desktop\projeto\DataSet')



model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2))

model.summary()


model.compile(loss='mse',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
#em duvida se vou colocar mae

history=model.fit_generator(train_it,
      steps_per_epoch=20,
      epochs=10,
      validation_data=val_it,
      validation_steps=20)

Using TensorFlow backend.


Numero de ficheiros Validation: 400
Numero de ficheiros Training: 400
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         


NotImplementedError: 